# Fase Beta de la entrega del reto
### ***Secuencia***
1. Generar frecuencias de ventas y simular una venta de 105 clientes.
2. Tomar los volúmenes de las compras de los clientes para determinar la cantidad de camiones necesarios.
3. Sabiendo los camiones, utilizar k-medoids para hacer los mini-tcps.
4. Resolver los mtcps.

### ***Librerías***

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import math

from scipy.stats import poisson
from scipy.stats import poisson
from scipy.stats import chisquare


from ortools.linear_solver import pywraplp

from scipy.spatial.distance import cdist

import time

# Lo que se corre 1 vez solo

In [2]:
compras = pd.read_excel('informacion_compra.xlsx')

In [3]:
compras.head(3)

,Producto,Unidades,Factura
0,48443,1,799186
1,42877,1,717106
2,48296,1,468125


In [4]:
facturas = []
productos = []
for _,i in compras.iterrows():
    facturas += [i[2]]*i[1]
for _,i in compras.iterrows():
    productos += [i[0]]*i[1]

compras = pd.DataFrame({
    "Factura": facturas,
    "Producto": productos
})

In [5]:
df = compras.groupby(['Factura']).size().reset_index(name='Frequency').Frequency.value_counts()
df = df.reset_index()
df.columns = ['Pedidos', 'Frecuencia']
df['Frecuencia ln'] = np.log(df['Frecuencia'])
valor_medio = np.sum(df['Pedidos'] * df['Frecuencia ln']) / np.sum(df['Frecuencia ln'])
df['Poisson'] = poisson.pmf(df['Pedidos'], valor_medio)
df['valores_esperados'] = df['Poisson'] * np.sum(df['Frecuencia ln'])
df.head(3)

,Pedidos,Frecuencia,Frecuencia ln,Poisson,valores_esperados
0,1,19235,9.864487,0.011672,0.783311
1,2,3546,8.173575,0.036706,2.463277
2,3,1024,6.931472,0.076953,5.164178


In [6]:
df2 = compras.groupby(['Producto']).size().reset_index(name='Frequency').Frequency.value_counts()
df2 = df2.reset_index()
df2.columns = ['Producto', 'Frecuencia']
df2['Frecuencia ln'] = np.log(df2['Frecuencia'])
valor_medio = np.sum(df2['Producto'] * df2['Frecuencia ln']) / np.sum(df2['Frecuencia ln'])
df2['Poisson'] = poisson.pmf(df2['Producto'], valor_medio)
df2['valores_esperados'] = df2['Poisson'] * np.sum(df2['Frecuencia ln'])
df2.head(3)

,Producto,Frecuencia,Frecuencia ln,Poisson,valores_esperados
0,1,1793,7.491645,1.565792e-11,2.886239e-09
1,2,779,6.658011,2.209339e-10,4.072494e-08
2,3,414,6.025866,2.078257e-09,3.830870e-07


In [7]:
def montecarlo(df, n):  
    resultados = []
    for _ in range(n):
        num = np.random.rand()
        for j in range(len(df)):
            if df[j][3] <= num < df[j][4]:
                resultados.append(j)
            
    return resultados

In [8]:
pedidos = df[['Pedidos','Poisson']]
pedidos['Acumulado'] = pedidos.Poisson.cumsum()
pedidos['Inferior'] = [0] + pedidos['Acumulado'].tolist()[:-1]
pedidos['Superior'] = pedidos['Acumulado']
pedidos = pedidos.to_numpy()

C:\Users\david\AppData\Local\Temp\ipykernel_18112\3932492861.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedidos['Acumulado'] = pedidos.Poisson.cumsum()
C:\Users\david\AppData\Local\Temp\ipykernel_18112\3932492861.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedidos['Inferior'] = [0] + pedidos['Acumulado'].tolist()[:-1]


In [9]:
#num_pedidos = montecarlo(pedidos,df.Frecuencia.sum())
num_pedidos = montecarlo(pedidos,compras.size)


In [10]:
producto = df2[['Producto','Poisson']]
producto['Acumulado'] = producto.Poisson.cumsum()
producto['Inferior'] = [0] + producto['Acumulado'].tolist()[:-1]
producto['Superior'] = producto['Acumulado']
producto = producto.to_numpy()

C:\Users\david\AppData\Local\Temp\ipykernel_18112\3891866760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  producto['Acumulado'] = producto.Poisson.cumsum()
C:\Users\david\AppData\Local\Temp\ipykernel_18112\3891866760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  producto['Inferior'] = [0] + producto['Acumulado'].tolist()[:-1]


In [11]:
dp = pd.read_csv('info_productos.csv')
dp.columns = ['Producto', 'Volumen']
nuevo_registro = {"Producto": 0, "Volumen": 0}
dp = pd.concat([pd.DataFrame([nuevo_registro]), dp], ignore_index=True)
volumenes = dp.to_numpy()[:,1]

In [12]:
len(volumenes)

53633

In [13]:
path = "https://raw.githubusercontent.com/gerardoramc/AlgoritmoDatos/Gerardo/distancias_tiempos2.csv"
df = pd.read_csv(path)
df.head(3)

,Distance,Duration
0,0.00,0:00
1,15.26,0:22
2,14.93,0:17


In [14]:
df['Duration'] = pd.to_datetime(df['Duration'])
df['Duration'] = df['Duration'].dt.strftime('%H:%M')
df['Duration']=df['Duration'].astype('string')

In [15]:
def convert_to_seconds(time_str):
    hours, minutes = map(int, time_str.split(':'))
    total_seconds = hours * 3600 + minutes * 60
    return total_seconds

# Apply the conversion function to the DataFrame column
df['Segundos'] = df['Duration'].apply(convert_to_seconds)

print(df)

       Distance Duration  Segundos
0         0.000    00:00         0
1        15.260    00:22      1320
2        14.930    00:17      1020
3        23.420    00:33      1980
4         9.490    00:12       720
...         ...      ...       ...
11231    38.365    00:44      2640
11232     9.674    00:24      1440
11233    12.298    00:27      1620
11234     5.247    00:17      1020
11235     0.000    00:00         0

[11236 rows x 3 columns]


In [16]:
df.to_numpy()

array([[0.0, '00:00', 0],
       [15.26, '00:22', 1320],
       [14.93, '00:17', 1020],
       ...,
       [12.298, '00:27', 1620],
       [5.247, '00:17', 1020],
       [0.0, '00:00', 0]], dtype=object)

In [17]:
rangos = np.arange(0,11236,106)
rangos

array([    0,   106,   212,   318,   424,   530,   636,   742,   848,
         954,  1060,  1166,  1272,  1378,  1484,  1590,  1696,  1802,
        1908,  2014,  2120,  2226,  2332,  2438,  2544,  2650,  2756,
        2862,  2968,  3074,  3180,  3286,  3392,  3498,  3604,  3710,
        3816,  3922,  4028,  4134,  4240,  4346,  4452,  4558,  4664,
        4770,  4876,  4982,  5088,  5194,  5300,  5406,  5512,  5618,
        5724,  5830,  5936,  6042,  6148,  6254,  6360,  6466,  6572,
        6678,  6784,  6890,  6996,  7102,  7208,  7314,  7420,  7526,
        7632,  7738,  7844,  7950,  8056,  8162,  8268,  8374,  8480,
        8586,  8692,  8798,  8904,  9010,  9116,  9222,  9328,  9434,
        9540,  9646,  9752,  9858,  9964, 10070, 10176, 10282, 10388,
       10494, 10600, 10706, 10812, 10918, 11024, 11130])

In [18]:
def remove_first_row_and_column(input_array):
    if input_array.shape[0] <= 1 or input_array.shape[1] <= 1:
        raise ValueError("Input array must have at least 2 rows and 2 columns.")

    new_array = input_array[1:, 1:]
    return new_array


In [19]:
def k_medoids(X, k, max_iters=100):
    num_samples, num_features = X.shape
    medoids_indices = np.random.choice(num_samples, k, replace=False)
    medoids = X[medoids_indices]

    for _ in range(max_iters):
        distances = cdist(X, medoids, metric='euclidean')
        cluster_assignments = np.argmin(distances, axis=1)

        for i in range(k):
            cluster_points = X[cluster_assignments == i]
            cluster_distances = distances[cluster_assignments == i][:, i]
            new_medoid_index = np.argmin(cluster_distances)
            medoids[i] = cluster_points[new_medoid_index]

    return medoids, cluster_assignments


In [20]:
cuadrada = []
for i in range(len(rangos)-1):
    cuadrada.append(list((df['Segundos'][rangos[i]:rangos[i+1]])))
result_array = np.append(np.array(cuadrada), [(df['Segundos'][rangos[-1]:])], axis = 0)

mapa = result_array 
result_array.shape

(106, 106)

## Filthy cycle

In [21]:
envios_np.shape

NameError: name 'envios_np' is not defined

In [27]:
trucks_per_iter = 0
iters = 10
start_time=time.time()

for i in range(iters):

        envios = [np.array(montecarlo(producto, num)) for num in montecarlo(pedidos,126) if num!=0]
        envios_np = envios[:106]

        dimensiones = []
        for pedido in envios_np:
            acumulado = 0
            for i in pedido:
                acumulado += volumenes[i]
            dimensiones.append(round(acumulado,4))

        result_array = mapa
        # Ejemplo de uso

        result_array_no_warehouse = remove_first_row_and_column(result_array)
        X = result_array_no_warehouse

        k = 6
        medoids, cluster_assignments = k_medoids(X, k)

        #print("Medoides finales:")
        #print(medoids)

        clusters = {}
        for i in range(k):
            cluster_points = np.where(cluster_assignments == i)[0]
            clusters[i] = cluster_points

        #print("Nodos en cada cluster:")
        '''for cluster_id, node_indices in clusters.items():
            print(f"\n***Cluster {cluster_id+1}*** \nCantidad: {len(node_indices)}\nNodos: {node_indices+1}\n")'''



        vol_por_clus = {}

        for clave, indices in clusters.items():
            pesos = [dimensiones[indice] for indice in indices]
            vol_por_clus[clave] = pesos


        multiplier = 1
        trucks_used = 0

        for clave, VOLU in vol_por_clus.items():
            incomplete = True
            while incomplete:
                data = {}
                data["weights"] = VOLU
                #data["values"] = len(data["weights"]) * [1]
                data["values"] =  VOLU
                sum_of_all_weights = sum(data["weights"])

                assert len(data["weights"]) == len(data["values"])
                data["num_items"] = len(data["weights"])
                data["all_items"] = range(data["num_items"])

                data["bin_capacities"] = [27] * multiplier
                data["num_bins"] = len(data["bin_capacities"])
                data["all_bins"] = range(data["num_bins"])

                solver = pywraplp.Solver.CreateSolver("SCIP")

                # x[i, b] = 1 if item i is packed in bin b.
                x = {}
                for i in data["all_items"]:
                    for b in data["all_bins"]:
                        x[i, b] = solver.BoolVar(f"x_{i}_{b}")

                # Each item is assigned to at most one bin.
                for i in data["all_items"]:
                    solver.Add(sum(x[i, b] for b in data["all_bins"]) <= 1)

                # The amount packed in each bin cannot exceed its capacity.
                for b in data["all_bins"]:
                    solver.Add(
                        sum(x[i, b] * data["weights"][i] for i in data["all_items"])
                        <= data["bin_capacities"][b])

                # Each bin must contain at least 2 items
                for b in data["all_bins"]:
                    solver.Add(sum(x[i, b] for i in data["all_items"]) <= 13)

                 # Each bin must contain at most 20 items
                '''for b in data["all_bins"]:
                    solver.Add(sum(x[i, b] for i in data["all_items"]) <= 20)'''

                # Maximize total value of packed items.
                objective = solver.Objective()
                for i in data["all_items"]:
                    for b in data["all_bins"]:
                        objective.SetCoefficient(x[i, b], data["values"][i])
                objective.SetMaximization()





                start_time=time.time()
                status = solver.Solve()

                if status == pywraplp.Solver.OPTIMAL:

                    unused_items = [i for i in data["all_items"] if all(x[i, b].solution_value() == 0 for b in data["all_bins"])]


                    used_bins = [b for b in data["all_bins"] if any(x[i, b].solution_value() > 0 for i in data["all_items"])]
                    if len(unused_items) != 0:
                        multiplier += 1

                    if len(unused_items) == 0:

                        trucks_used += len(used_bins)
                        incomplete = False



                else:
                    print("No existe solución óptima.")


        #print(f"Camiones totales usados: {trucks_used}")
        trucks_per_iter += trucks_used
        
    

end_time=time.time()-start_time
print(f"{end_time} segundos")

print(f"Camiones promedio: {math.ceil(trucks_per_iter/iters)}")

KeyboardInterrupt: 

In [ ]:
len(dimensiones)

In [ ]:
envios = [np.array(montecarlo(producto, num)) for num in montecarlo(pedidos,110) if num!=0]
envios_np = envios[:106]

In [ ]:
dimensiones = []
for pedido in envios_np:
    acumulado = 0
    for i in pedido:
        acumulado += volumenes[i]
    dimensiones.append(round(acumulado,4))

In [ ]:

# Ejemplo de uso

result_array_no_warehouse = remove_first_row_and_column(result_array)
X = result_array_no_warehouse

k = 6
medoids, cluster_assignments = k_medoids(X, k)

#print("Medoides finales:")
#print(medoids)

clusters = {}
for i in range(k):
    cluster_points = np.where(cluster_assignments == i)[0]
    clusters[i] = cluster_points

print("Nodos en cada cluster:")
for cluster_id, node_indices in clusters.items():
    print(f"\n***Cluster {cluster_id+1}*** \nCantidad: {len(node_indices)}\nNodos: {node_indices+1}\n")



In [ ]:
clusters

In [ ]:
vol_por_clus = {}

for clave, indices in clusters.items():
    pesos = [dimensiones[indice] for indice in indices]
    vol_por_clus[clave] = pesos

vol_por_clus

In [ ]:
multiplier = 1
trucks_used = 0
start_time=time.time()
for clave, volumenes in vol_por_clus.items():
    print(f"**********CLUSTER # {clave + 1 }**********\n")
    incomplete = True
    while incomplete:
        data = {}
        data["weights"] = volumenes
        #data["values"] = len(data["weights"]) * [1]
        data["values"] =  volumenes
        sum_of_all_weights = sum(data["weights"])

        assert len(data["weights"]) == len(data["values"])
        data["num_items"] = len(data["weights"])
        data["all_items"] = range(data["num_items"])

        data["bin_capacities"] = [27] * multiplier
        data["num_bins"] = len(data["bin_capacities"])
        data["all_bins"] = range(data["num_bins"])

        solver = pywraplp.Solver.CreateSolver("SCIP")

        # x[i, b] = 1 if item i is packed in bin b.
        x = {}
        for i in data["all_items"]:
            for b in data["all_bins"]:
                x[i, b] = solver.BoolVar(f"x_{i}_{b}")

        # Each item is assigned to at most one bin.
        for i in data["all_items"]:
            solver.Add(sum(x[i, b] for b in data["all_bins"]) <= 1)

        # The amount packed in each bin cannot exceed its capacity.
        for b in data["all_bins"]:
            solver.Add(
                sum(x[i, b] * data["weights"][i] for i in data["all_items"])
                <= data["bin_capacities"][b])
        
        # Each bin must contain at least 2 items
        '''for b in data["all_bins"]:
            solver.Add(sum(x[i, b] for i in data["all_items"]) >= 2)'''
            
         # Each bin must contain at most 20 items
        '''for b in data["all_bins"]:
            solver.Add(sum(x[i, b] for i in data["all_items"]) <= 20)'''
        
        # Maximize total value of packed items.
        objective = solver.Objective()
        for i in data["all_items"]:
            for b in data["all_bins"]:
                objective.SetCoefficient(x[i, b], data["values"][i])
        objective.SetMaximization()





        start_time=time.time()
        status = solver.Solve()

        if status == pywraplp.Solver.OPTIMAL:

            unused_items = [i for i in data["all_items"] if all(x[i, b].solution_value() == 0 for b in data["all_bins"])]


            used_bins = [b for b in data["all_bins"] if any(x[i, b].solution_value() > 0 for i in data["all_items"])]
            if len(unused_items) != 0:
                multiplier += 1

            if len(unused_items) == 0:


                print(f"Número de camiones requeridos: {len(used_bins)}\n")

                print(f"Valor total empaquetado: {round(objective.Value(), 2)}\n")

                max_value_bin = None
                max_value = 0
                total_weight = 0
                total_value = 0
                for b in used_bins:
                    print(f"*** CAMIÓN # {b+1} ***")
                    print(f'Capacidad: {data["bin_capacities"][b]}')
                    bin_weight = 0
                    bin_value = 0
                    packed_items = []
                    for i in data["all_items"]:
                        if x[i, b].solution_value() > 0:
                            packed_items.append(str(clusters[clave][i]+1))
                            bin_weight += data["weights"][i]
                            bin_value += data["values"][i]
                    print(f"Clientes empacados: {', '.join(packed_items)}")
                    print(f"Volumen empaquetado en el camión: {round(bin_weight, 4)}\n")
                    #print(f"Valor empaquetado del camión: {bin_value}\n")
                    total_value += bin_value
                    total_weight += bin_weight
                    if bin_value > max_value:
                        max_value = bin_value
                        max_value_bin = b

                trucks_used += len(used_bins)
                incomplete = False

            # Print solution for the new bin (if exists)



        else:
            print("No existe solución óptima.")
end_time=time.time()-start_time

print(f"Camiones totales usados: {trucks_used}")
print(f"{end_time} segundos")

In [ ]:
for i in data["all_items"]:
    if x[i, b].solution_value() > 0:
        data[]
        print(i)

In [ ]:
clusters[0][0]

In [ ]:
used_bins

### ***Mochila sobre medoides***

In [ ]:
for clus in clusters:
    data = {}
    data["weights"] = [48, 30, 42, 36, 36, 48, 42, 42, 36, 24, 30, 30, 42, 36] 
    data["values"] = [10, 30, 25, 50, 35, 30, 15, 40, 30, 35, 45, 10, 20, 36]
    sum_of_all_weights = sum(data["weights"])

    assert len(data["weights"]) == len(data["values"])
    data["num_items"] = len(data["weights"])
    data["all_items"] = range(data["num_items"])

    data["bin_capacities"] = [100, 100, 100, 100, 100, 100]
    data["num_bins"] = len(data["bin_capacities"])
    data["all_bins"] = range(data["num_bins"])

In [ ]:
mini_tcps = []
for i in clusters:
    i = list(clusters[i]) + [0]
    mini_tcps.append(result_array[i,:][:,i])

In [ ]:
mini_tcps[1]

### Mochila Approach

In [ ]:
data = {}
data["weights"] = dimensiones
data["values"] = [1] * 100
sum_of_all_weights = sum(data["weights"])

assert len(data["weights"]) == len(data["values"])
data["num_items"] = len(data["weights"])
data["all_items"] = range(data["num_items"])

data["bin_capacities"] = [27] * 4
data["num_bins"] = len(data["bin_capacities"])
data["all_bins"] = range(data["num_bins"])

In [ ]:
solver = pywraplp.Solver.CreateSolver("SCIP")

In [ ]:
# x[i, b] = 1 if item i is packed in bin b.
x = {}
for i in data["all_items"]:
    for b in data["all_bins"]:
        x[i, b] = solver.BoolVar(f"x_{i}_{b}")

In [ ]:
# Each item is assigned to at most one bin.
for i in data["all_items"]:
    solver.Add(sum(x[i, b] for b in data["all_bins"]) <= 1)

# The amount packed in each bin cannot exceed its capacity.
for b in data["all_bins"]:
    solver.Add(
        sum(x[i, b] * data["weights"][i] for i in data["all_items"])
        <= data["bin_capacities"][b])
    
'''# Each item has to be shipped
for i in data["all_items"]:
    solver.Add(sum(x[i, b] for b in data["all_bins"]) == 1)'''

In [ ]:
# Maximize total value of packed items.
objective = solver.Objective()
for i in data["all_items"]:
    for b in data["all_bins"]:
        objective.SetCoefficient(x[i, b], data["values"][i])
objective.SetMaximization()

In [ ]:
start_time=time.time()
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    
    used_bins = [b for b in data["all_bins"] if any(x[i, b].solution_value() > 0 for i in data["all_items"])]
    
    print(f"Número de camiones requeridos: {len(used_bins)}\n")
    
    print(f"Valor total empaquetado: {round(objective.Value(), 2)}\n")
    
    max_value_bin = None
    max_value = 0
    total_weight = 0
    total_value = 0
    for b in used_bins:
        print(f"*** CAMIÓN # {b+1} ***")
        print(f'Capacidad: {data["bin_capacities"][b]}')
        bin_weight = 0
        bin_value = 0
        packed_items = []
        for i in data["all_items"]:
            if x[i, b].solution_value() > 0:
                packed_items.append(str(i))
                bin_weight += data["weights"][i]
                bin_value += data["values"][i]
        print(f"Productos empacados: {', '.join(packed_items)}")
        print(f"Volumen empaquetado en el camión: {round(bin_weight, 4)}\n")
        #print(f"Valor empaquetado del camión: {bin_value}\n")
        total_value += bin_value
        total_weight += bin_weight
        if bin_value > max_value:
            max_value = bin_value
            max_value_bin = b
    
    print(f"Volumen total empacado: {round(total_weight, 4)}")
    print(f"Volumen no empacado: {round(sum_of_all_weights - total_weight, 4)}")
    
    # Print solution for the new bin (if exists)
    
    unused_items = [i for i in data["all_items"] if all(x[i, b].solution_value() == 0 for b in data["all_bins"])]
    if unused_items:
        print(f"\n*** CAMIÓN DE SOBRANTES # {len(used_bins)+1} ***")
        bin_weight = 0
        bin_value = 0
        packed_items = []
        for i in unused_items:
            packed_items.append(str(i))
            bin_weight += data["weights"][i]
            bin_value += data["values"][i]
        
        print(f"Productos: {', '.join(packed_items)}")
        print(f"Volumen empaquetado en el camión de sobrantes: {round(bin_weight, 4)}")
        print(f"Valor empaquetado del camión de sobrantes: {bin_value}\n")

    else:
        print("No hay camión extra.")
        
    # Print solution for the bin with the highest value
    if max_value_bin is not None:
        print(f"Camión más valioso: {max_value_bin+1} (Valor: {round(max_value, 4)})")
        
else:
    print("No existe solución óptima.")
end_time=time.time()-start_time
print(f"{end_time} segundos")

In [ ]:
len(dimensiones)

In [ ]:
y = [1]*100
len(y)

In [ ]:
for clus in clusters:
    print("Hi")